In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns

In [ ]:
from tensorflow import keras

In [ ]:
call_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/call_black_scholes.csv')
put_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/put_black_scholes.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#WITH SVOL
call_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/Call_with_svol.csv')
put_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/Put_with_svol.csv')

In [ ]:
call_df=call_df.iloc[:,1:]
put_df=put_df.iloc[:,1:]

In [ ]:
call_df=call_df.set_index(['Date'])
put_df=put_df.set_index(['Date'])

In [ ]:
call_df=call_df[call_df['Close']>1]

In [ ]:
Deep_itm=call_df[call_df['Adj Close']-call_df['Strike Price']>3000]
itm=call_df[(call_df['Adj Close']-call_df['Strike Price']>0) & (call_df['Adj Close']-call_df['Strike Price']<3000)]
otm=call_df[call_df['Adj Close']<call_df['Strike Price']]

In [ ]:
def linear_regression(df):
  x=np.array(df['Close']).reshape(-1,1)
  y=np.array(df['bs_call']).reshape(-1,1)
  reg=LinearRegression().fit(x,y)
  mse= mean_absolute_error(x,y)
  return reg.score(x,y),ms

In [ ]:
def regression(df):
  sns.scatterplot(x=df['Close'],y=df['bs_call'],hue=df['Expiration'])

In [ ]:
call_df.head()

In [ ]:
call_df["svol"]*=np.sqrt(252)

In [ ]:
def split_data(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Expiration','Adj Close','Price','volatility']],df['Close'],test_size=0.3,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [ ]:
def split_data_bs(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Expiration','Adj Close','Price']],df['bs_call'],test_size=0.3,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test=split_data(call_df)

In [ ]:
call_X_train_bs, call_X_test_bs, call_y_train_bs, call_y_test_bs=split_data_bs(call_df)


In [ ]:
n_units = 400
layers = 5
n_batch = 100
n_epochs = 20

In [ ]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(LeakyReLU())

model.add(Dense(1, activation='relu'))
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='mse', optimizer=opt)

In [ ]:
model.summary()

In [ ]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.3,
                    callbacks=[TensorBoard()],
                    verbose=1)

In [ ]:
call_y_pred = model.predict(call_X_test)

In [ ]:
call_y_pred.shape

In [ ]:
x=np.array(call_y_test).reshape(-1,1)
y=np.array(call_y_pred).reshape(-1,1)
reg=LinearRegression().fit(x,y)
mae= mean_absolute_error(x,y)

In [ ]:
print(reg.score(x,y),mae)

In [ ]:
plt.scatter(x,y)

In [ ]:
from sklearn.metrics import r2_score
r2_score(call_y_test,call_y_pred)

In [ ]:
r2_score(call_y_test,call_y_test_bs)